# TAREA DE EJERCICIO

Actualiza el proyecto del día 1 para resumir una página web y utilizar un modelo de código abierto que se ejecute localmente a través de Ollama en lugar de OpenAI

Podrás utilizar esta técnica para todos los proyectos posteriores si prefiere no utilizar API de pago (closed source).

**Beneficios:**
1. Sin cargos por API: código abierto
2. Los datos no salen de su ordenador

**Desventajas:**
1. Tiene mucha menos potencia (parámetros) que el modelo Frontier

## Resumen de la instalación de Ollama

¡Simplemente visita [ollama.com](https://ollama.com) e instálalo!

Una vez que hayas terminado, el servidor ollama ya debería estar ejecutándose localmente.
Si entras en:
[http://localhost:11434/](http://localhost:11434/)

Debería ver el mensaje `Ollama se está ejecutando`.

De lo contrario, abre una nueva Terminal (Mac) o Powershell (Windows) e introduce `ollama serve`.
Luego, intenta entrar em [http://localhost:11434/](http://localhost:11434/) nuevamente.

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [6]:
# Constantes

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "gemma3:1b"

In [7]:
# Crea una lista de mensajes utilizando el mismo formato que usamos para OpenAI

messages = [
    {"role": "user", "content": "Describe algunas de las aplicaciones comerciales de la IA generativa."}
]

In [8]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [9]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

La IA generativa está revolucionando una amplia gama de industrias y aplicaciones comerciales. Aquí te presento un resumen de algunas de las más destacadas, categorizadas por área:

**1. Marketing y Ventas:**

* **Generación de contenido:**
    * **Redacción de textos:** La IA genera descripciones de productos, publicaciones de blog, correos electrónicos de marketing, scripts de video, etc.  Esto ahorra tiempo y recursos al equipo de marketing.
    * **Generación de imágenes y videos:** Crea imágenes, ilustraciones y videos de alta calidad a partir de descripciones textuales, lo que facilita la producción de contenido visual.
    * **Personalización del contenido:** Crea mensajes y ofertas personalizadas en función de los datos del cliente.
* **Chatbots y Asistentes Virtuales:**
    * **Chatbots inteligentes:**  Crean experiencias de conversación más fluidas y conversacionales, mejorando la atención al cliente y la generación de leads.
    * **Asistentes virtuales personalizados:**  Au

# Presentación del paquete ollama

Ahora haremos lo mismo, pero utilizando el elegante paquete de Python ollama en lugar de una llamada HTTP directa.

En esencia, se realiza la misma llamada que se indicó anteriormente al servidor ollama que se ejecuta en localhost:11434

In [10]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

La IA generativa ha avanzado a un ritmo vertiginoso y está abriendo un abanico de aplicaciones comerciales en diversos sectores. Aquí te presento algunas de las más destacadas, divididas por áreas:

**1. Marketing y Ventas:**

* **Generación de contenido:**
    * **Textos:** Crea contenido de marketing (descripciones de productos, publicaciones para redes sociales, correos electrónicos, artículos de blog, etc.) a partir de una simple descripción inicial.
    * **Imágenes y vídeos:** Genera imágenes, vídeos cortos, elementos visuales y animaciones para campañas publicitarias y marketing.
    * **Guiones:** Crea guiones para vídeos y anuncios.
* **Personalización del marketing:**  Diseña e-mails, mensajes y experiencias personalizadas para segmentos específicos de clientes, optimizando la efectividad de las campañas.
* **Generación de scripts:** Crea scripts para vídeos explicativos, vídeos de testimonios y otros contenidos promocionales.
* **Creación de Avatar y personajes:** Desarrolla

# AHORA el ejercicio para ti

Toma el código del día 1 e incorpóralo aquí para crear un resumidor de sitios web que use Llama 3.2 ejecutándose localmente en lugar de OpenAI

In [15]:
import ollama

MODEL = "gemma3:1b"


class Website:
    def __init__(self, url):
        self.url=url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No tiene título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)



In [42]:
system_prompt = "Eres un asistente que analiza el contenido de un sitio web \
 y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. \
Responder en Markdown."

def user_prompt_for(website):
    user_prompt = f"Estás viendo un sitio web titulado {website.title}"
    user_prompt += "\nEl contenido de este sitio web es el siguiente; \
    proporciona un breve resumen de este sitio web en formato Markdown. \
    Si el contenido no está en español, traduce el resumen automáticamente al español.\n\n"
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']


def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://www.anthropic.com/engineering/building-effective-agents")

Okay, here’s a brief summary of the provided text, suitable for a quick understanding:

Anthropic is developing AI agents – systems that can autonomously operate and complete complex tasks – through a series of principles and design choices focused on simplicity, transparency, and human oversight. The text outlines a process of building these agents, starting with simple prompts and gradually incorporating more sophisticated tools and frameworks. Key areas of focus include:  understanding the nuances of agent-computer interfaces (ACI), managing tool-specific error handling, and prioritizing clear documentation and testing.  The document emphasizes best practices for tool development, including careful parameterization and defining clear boundaries, as well as the importance of creating trust-worthy agent-computer interfaces.  Finally, it highlights key design choices for building robust and reliable AI agents, including integrating feedback loops and careful consideration of risk and complexity, all while emphasizing the importance of continuous learning and iteration.

# Contribution: Modo batch y traduccion a español

In [28]:
# Nuevo: Modo batch

def batch_summarize(urls):
    """
    Procesa multiples urls y devuelve un diccionario {url: resumen}.
    Traduce a español si es necesario.
    """
    summaries = {}
    for url in urls:
        try:
            summaries[url] = summarize(url)
        except Exception as e:
            summaries[url] = f"Error al procesar {url}: {e}"
    return summaries



In [40]:
urls = [
    "https://ir.uitm.edu.my/id/eprint/29153/",
    "https://www.worldvaluessurvey.org/wvs.jsp",
    "https://www.wonderfulcopenhagen.com/cll",
    "https://www.bellingcat.com"
]

def display_summaries(urls):
    summaries = batch_summarize(urls)
    for url, summary in summaries.items():
        text = f"""
### [{url}]({url})

{summary}

---
"""
        display(Markdown(text))

In [41]:
display_summaries(urls)


### [https://ir.uitm.edu.my/id/eprint/29153/](https://ir.uitm.edu.my/id/eprint/29153/)

El resumen de la publicación "Key success factors toward MICE industry: A systematic literature review / Muhammad Saufi Anas ... [et al.]" (JTHCA, 2020) es el siguiente:

La publicación investiga los factores críticos que contribuyen al éxito de la industria MICE. El trabajo realiza una revisión sistemática de literatura sobre 39 publicaciones sobre este tema, evaluando las estrategias de las empresas que operan en este sector.  Los hallazgos revelan que factores como la motivación de los viajeros de negocios, la percepción sobre la industria, la motivación de los asistentes a eventos y la importancia de los indicadores clave de rendimiento, la satisfacción de los asistentes y las estrategias de marketing, las tendencias tecnológicas y los desafíos enfrentados por la industria son cruciales. El estudio identifica la importancia de la sostenibilidad de la industria.  El trabajo ofrece información útil para comprender las estrategias de desarrollo de la industria, los indicadores clave de éxito, los desafíos y las formas de impulsar la sostenibilidad de la industria MICE.


---



### [https://www.worldvaluessurvey.org/wvs.jsp](https://www.worldvaluessurvey.org/wvs.jsp)

Okay, here’s a summary of the WVS Database content, formatted as Markdown:

**The World Values Survey (WVS) Database Summary**

The WVS Database is a comprehensive resource documenting the findings of the Ronald F. Inglehart Best Book Award on Political Culture and Values, focusing on comparative cross-cultural research. It provides details on:

*   **The Award Process:**  Details about the award criteria, nominations, and selection process for the 2025 award.
*   **Recent Webinars:**  A detailed timeline of recent webinars, including:
    *   **July 15th:**  Discussion of Autocratic Modernity and Psychological Cracks in a Dystopian Model.
    *   **June 27th:** Global Attitudes Toward Abortion: Insights from the World Values Survey.
    *   **June 18th:** Ideological Extremism and Polarization on a Global Scale.
    *   **April 25th:** Elite-Citizen Gap in International Organization Legitimacy.
    *   **April 08th:**  The Persistence of Traditional Values and the Limited Global Appeal of Democracy.
    *   **March 25th:**  Predicting Homonegativity in Southeast Asian Countries – Using Survey Data.
    *   **May 20th:**  Measuring National Parochialism and Explaining its Individual Variations Using Survey Data.
*   **Ongoing Research & Events:** Information on ongoing research projects and events hosted by the WVSA, including:
    *   **Social Networks:**  A Twitter feed highlighting discussions about the WVS research.
    *   **Latest News & Events:** A calendar of events and news updates.
    *   **Events:**  Details of the WVSA’s secretariat and headquarters – Christian Haerpfer, Christian Diaz-Medrano, and Jaime Diez-Medrano.
    *   **Data Archive:**  Information about the database's archival materials.

The database primarily centers around the research and analysis generated by the Ronald F. Inglehart Best Book Award, exploring comparative cross-cultural phenomena and political attitudes.

---

Would you like me to elaborate on any specific section or aspect of the database content?

---



### [https://www.wonderfulcopenhagen.com/cll](https://www.wonderfulcopenhagen.com/cll)

El Copenhagen Legacy Lab es un proyecto de la organización Wonderful Copenhagen que se centra en impulsar el impacto a largo plazo de las conferencias y eventos.  El laboratorio, impulsado por una investigación doctoral, ha desarrollado un marco de legado basado en datos y experiencia, y ha recibido varios reconocimientos, incluyendo el “2024 GDS-Movement” y el “#MEET4IMPACT” Impact Award.  El objetivo principal es apoyar a las asociaciones internacionales en alcanzar sus objetivos estratégicos, al mismo tiempo que respaldan las prioridades nacionales y las fortalezas locales para generar valor social a nivel local y global.  El laboratorio ofrece servicios gratuitos, pero la participación y los recursos deben priorizarse por parte de las asociaciones y/o los hostes.  En esencia, el lab busca facilitar la colaboración entre asociaciones internacionales y partes interesadas locales, identificando intereses comunes y delineando acciones concretas para el legado de los eventos.  El sitio web proporciona testimonios de asociaciones que han utilizado la iniciativa, y ofrece enlaces a la página de la Convention Bureau, la página de la Card de Copenhague y la página de Wonderful Copenhagen.

---



### [https://www.bellingcat.com](https://www.bellingcat.com)

Okay, here's a Markdown summary of the Bellingcat website, focusing on the core information presented:

**Bellingcat Overview:**

Bellingcat is an online investigative journalism organization dedicated to uncovering wrongdoing and holding power accountable.  They focus on investigations related to:

*   **Ukraine:**  Specifically, tracking traffic stops involving delivery riders and workers, investigations related to immigration, and analyzing misinformation campaigns.
*   **Conflict:**  Focuses on analyzing events and data related to conflict zones, particularly in Ukraine and surrounding regions.
*   **China:** Investigating potential connections between Chinese fentanyl smuggling networks and Japan.
*   **Global Affairs:**  Covers a wide range of global issues, including misinformation, legal accountability, and geopolitical investigations.

**Key Activities & Features:**

*   **Investigations:**  They publish detailed investigations, including data analysis, geospatial mapping, and source material.
*   **Guides:** Provides accessible resources to help researchers and journalists with open-source investigation techniques.
*   **Workshops:** Offers online training workshops on open-source tools and techniques.
*   **Community:** Encourages collaboration through Discord and other platforms.
*   **Tools:**  Offers open-source tools for researchers to utilize.
*   **Newsletter:**  Delivers timely content and updates.
*   **Collaboration:**  Facilitates a global community of researchers and journalists.
*   **Data Journalism:**  Focuses on visualising data and uncovering stories.

**Recent Highlights:**

*   Discovered a key administrator behind an AI porn site.
*   Confirmed a Dutch political party’s involvement in calling for Canada’s extradition.
*   Won the AI Neuharth Innovation Award.

**Overall Purpose:**

Bellingcat aims to empower investigative journalism and transparency by providing resources, tools, and a collaborative community for uncovering wrongdoing and holding power accountable, particularly in politically sensitive areas like Ukraine and China.

---

Let me know if you'd like me to elaborate on any specific aspect or provide a more detailed breakdown!

---
